# NBA Data Analysis

The data used in this notebook was downloaded from [Kaggle](https://www.kaggle.com/drgilermo/nba-players-stats#Seasons_Stats.csv).  The original source of the data is [Basketball-reference](http://www.basketball-reference.com/).


## General Intro EDA

In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import pandas as pd
import numpy as np

from scipy import stats
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import plotly_express as px
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

data_url = "https://docs.google.com/spreadsheets/d/1m0jaYL1KGjxW1cKJUQxVTcPOnm7v7NZEBKRZADCmc68/export?format=csv"
nba = pd.read_csv(data_url, index_col=0)
nba.head()

Year           Player  Pos   Age   Tm     G  GS  MP  PER    TS%  ...  \
0  1950.0  Curly Armstrong  G-F  31.0  FTW  63.0 NaN NaN  NaN  0.368  ...   
1  1950.0     Cliff Barker   SG  29.0  INO  49.0 NaN NaN  NaN  0.435  ...   
2  1950.0    Leo Barnhorst   SF  25.0  CHS  67.0 NaN NaN  NaN  0.394  ...   
3  1950.0       Ed Bartels    F  24.0  TOT  15.0 NaN NaN  NaN  0.312  ...   
4  1950.0       Ed Bartels    F  24.0  DNN  13.0 NaN NaN  NaN  0.308  ...   

     FT%  ORB  DRB  TRB    AST  STL  BLK  TOV     PF    PTS  
0  0.705  NaN  NaN  NaN  176.0  NaN  NaN  NaN  217.0  458.0  
1  0.708  NaN  NaN  NaN  109.0  NaN  NaN  NaN   99.0  279.0  
2  0.698  NaN  NaN  NaN  140.0  NaN  NaN  NaN  192.0  438.0  
3  0.559  NaN  NaN  NaN   20.0  NaN  NaN  NaN   29.0   63.0  
4  0.548  NaN  NaN  NaN   20.0  NaN  NaN  NaN   27.0   59.0  

[5 rows x 52 columns]

<IPython.core.display.Javascript object>

Preprocessing from before:

If the impact for dropping NAs based on these columns is low, perform the drop:

In [3]:
nba = nba.drop(columns=["blank2", "blanl"])
nba = nba.dropna(subset=["Year", "Player", "Pos", "Tm"])

<IPython.core.display.Javascript object>

## Feature Engineering

We have a lot of useful data here, but most predictive models that we'll be looking at only like numeric data.  To still use our information we have to do a little bit of reformatting.

### One Hot Encoding / Dummy Encoding

For example, for team, we might "one-hot encode" (aka create dummy variables).  This is a way of creating a series of variables indicating True/False.

Create a dataframe that is a subset of the `nba` dataframe.  Only include in this subset:

* Columns: `PTS`, `Player`, & `Tm`
* Rows: a random selection of 15 rows (use 42 as the `random_state`)

In [15]:
# subset columns
nba_sub = nba[["PTS", "Player", "Tm"]]

# subset rows
nba_sub = nba_sub.sample(15, random_state=42)

nba_sub

PTS             Player   Tm
19756  191.0    Morris Peterson  NOH
20547   79.0        Earl Barron  TOT
9517   158.0         Eric White  LAC
12379  228.0        Bimbo Coles  GSW
6383   444.0        Alvin Scott  PHO
14879  774.0          Joe Smith  MIN
5892   607.0      Ollie Johnson  CHI
14520  131.0         Ryan Bowen  DEN
6879   212.0          Jeff Cook  CLE
2534   100.0            Jim Fox  CIN
4056   807.0     Nate Thurmond*  GSW
15031  193.0       Jud Buechler  DET
9765   258.0        Andrew Lang  PHO
24269  585.0  Langston Galloway  TOT
3570   412.0       Art Williams  BOS

<IPython.core.display.Javascript object>

Use `pd.get_dummies()` on the subset.

* What happened?
* What might we change about this and why?
* What does the `drop_first` argument of `pd.get_dummies()` do and why?

There are some issues that come up with using `pd.get_dummies` in a machine learning workflow.  For today, we'll stick with it due to its ease of use compared to more powerful options.

Using [`sklearn.preprocessing.OneHotEncoder`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) overcomes the issues that `pd.get_dummies` can run into, but it has a little more complex usage.

### Binary Encoding

Create a binary column named `is_old` that shows whether or not the `Year` variable is before 1980.

Create a binary column named `is_california` that shows whether or not a team is located in california.

In [17]:
ca_teams = ["LAL", "LAC", "GSW", "SAC"]



<IPython.core.display.Javascript object>

### Ordinal Encoding

Let's make up some data to be ordinal encoded.

* Using the `grades` list create a sample of 20 random letters
* Create a 1 column DataFrame from this sample

In [20]:
np.random.seed(42)

grades = ["A", "B", "C", 'D', 'F']
rand_grades = ____

grade_df = ____
grade_df.head()

<IPython.core.display.Javascript object>

Create a variable that is an ordinal encoding of grade.  Have `A` be 1 and `F` be 5.

### Scaling

Some methods we'll see are sensitive to our variables being on different scales.  For example, if you have variables for a person's height and their annual income, the height feature will have a much much smaller value than the income feature.  In some methods, this will lead to the income variable being a louder signal than the height variable.  Larger magnitude variables can end up drowning out smaller magnitude ones, and this can be an issue if we think height will be an important predictor.

To address this issue, we can scale the variables to have equal footing.  This won't change the shape of their distribution; this shape not changing means that the patterns within the variable aren't lost by scaling, the patterns are preserved, the values have just been standardized.

* Create a subset of the nba dataset that has the columns `PTS` and `Age`.
* Drop all NAs
* Use the pandas boxplot method on this resulting data.
* Plot these variables on a scatter plot.

In [26]:
nba_sub = nba[["PTS", "Age"]].dropna()
nba_sub.boxplot()
plt.show()

<IPython.core.display.Javascript object>

In [33]:
# fmt: off
data_scientists = ["Adam", "Emily", "Heather", "Hunter",
                   "Jon", "Michael", "Shobair", "Wyatt"]
# fmt: on

# Randomize order
np.random.shuffle(data_scientists)

n = len(data_scientists) // 2
print(f"Use StandardScaler: {data_scientists[:n]}")
print(f"Use MinMaxScaler: {data_scientists[n:]}")

Use StandardScaler: ['Jon', 'Heather', 'Michael', 'Wyatt']
Use MinMaxScaler: ['Hunter', 'Emily', 'Shobair', 'Adam']


<IPython.core.display.Javascript object>

In [45]:
# Pick your poison
scaler = StandardScaler()
# scaler = MinMaxScaler()

<IPython.core.display.Javascript object>

Use a scaler to scale the `PTS` and `Age` data.

In [40]:
# .fit() methods 'learn' something from your data
# They don't apply any of these learnings
# In the case of a scaler we have to call .transform
# Alternatively, we could use .fit_transform() to do 
# both of these things in one step
scaler.fit(nba_sub)

scaled = scaler.transform(nba_sub)
print(type(scaled))
print(scaled.shape)
print(scaled)

scaled_df = pd.DataFrame(scaled, columns=nba_sub.columns)
scaled_df.head()

<class 'numpy.ndarray'>
(24616, 2)
[[0.11367585 0.5       ]
 [0.06924795 0.42307692]
 [0.10871184 0.26923077]
 ...
 [0.00570861 0.07692308]
 [0.05956813 0.15384615]
 [0.07048896 0.03846154]]


PTS       Age
0  0.113676  0.500000
1  0.069248  0.423077
2  0.108712  0.269231
3  0.015637  0.230769
4  0.014644  0.230769

<IPython.core.display.Javascript object>

* Redo the boxplot from before
* Redo the scatterplot from before

What do you see? What's changed? What hasn't changed?

* Bonus: what attributes does your scaler have? What is the significance of these?
* Bonus Bonus: can you recreate this same scaling from scratch?